In [6]:
import uuid 
import pandas as pd

In [7]:
# read data
prod_data = pd.read_parquet("../data/training_data/production_data.parquet")
new_data = pd.read_parquet("../data/training_data/new_data/new_data.parquet")

In [3]:
# shuffle data
sprod_data = prod_data.sample(frac=1).reset_index(drop=True)
snew_data = new_data.sample(frac=1).reset_index(drop=True)
df_dict = {"prod_data" : sprod_data, 
           "new_data": snew_data}

In [4]:
splits = 3

for df_name, df in df_dict.items():
    lower_bound = 0
    upper_bound = int(round(len(df) / splits, 0))

    for i in range(splits):
        batch_name = f"{df_name}_batch_{i + 1}.parquet"
        subset = df.iloc[lower_bound:upper_bound].copy()
        subset.drop(columns="political_engagement", inplace=True)
        # link subject ID
        subset["subject_id"] = [str(uuid.uuid4()) for i in range(len(subset))]
        # updates slices
        lower_bound = upper_bound
        upper_bound += upper_bound 
        # export batches
        subset.to_parquet(f"../data/batches/{batch_name}", index=False)

In [5]:
# # generate dump_dates
# date_range = list(pd.date_range(start="2024-01-01", end="2024-12-31", freq="MS"))

# # write batches
# lower_bound = 0
# # define the slices (12 months)
# upper_bound = threshold = len(sdata) / 12

# for idx, vals in enumerate(date_range):
#     # assign dates by slices
#     sdata.loc[lower_bound:upper_bound, "dump_date"] = vals
#     # udpate index ranges
#     lower_bound += threshold
#     upper_bound += threshold or None

# # output batches
# for date in list(sdata.dump_date.unique()):
#     data_path = f"./data/batches/wvs7_{date.date()}.parquet"
#     batch_data = sdata[sdata.dump_date == date].copy()
#     batch_data.drop(columns=["dump_date", "political_engagement"], inplace=True)
#     batch_data["subject_id"] = [str(uuid.uuid4()) for i in range(len(batch_data))]
#     batch_data.to_parquet(data_path, index=False)